<a href="https://colab.research.google.com/github/AUT-Student/CN-HW1/blob/main/ComplexNetwork_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Class

In [8]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Edge():
  source: int
  destination: int

  # def sort(self):
  #   return Edge(min(self.destination, self.source),
  #               max(self.destination, self.source))
    
  def __eq__(self, other):
    return (self.source == other.source and self.destination == other.destination) or\
           (self.destination == other.source and self.source == other.destination)


In [25]:
class Graph():
  def __init__(self):
    self.nodes = set()
    self.edges = set()
    self.edge_list = dict()

  def add_node(self, node):
    self.nodes.add(node)

    if node not in self.edge_list:
      self.edge_list[node] = set()

  def add_edge(self, edge:Edge):
    self.add_node(edge.source)
    self.add_node(edge.destination)

    self.edge_list[edge.source].add(edge.destination)
    self.edge_list[edge.destination].add(edge.source)
    self.edges.add(edge)

  def remove_edge(self, edge:Edge):
    self.edge_list[edge.source].remove(edge.destination)
    self.edge_list[edge.destination].remove(edge.source)
    self.edges.remove(edge)

  def is_exist(self, edge:Edge):
    return edge in self.edges

  def get_number_nodes(self):
    return len(self.nodes)

  def get_number_edges(self):
    return len(self.edges)

  def get_cluster_coefficient_node(self, node):
    neighbours = self.edge_list[node]

    k_node = len(neighbours)
    e_node = 0

    if k_node <= 1: return 0

    for n1 in neighbours:
      for n2 in neighbours:
        if n1 < n2:
          if self.is_exist(Edge(n1, n2)):
            e_node += 1
    
    return (2 * e_node) / (k_node * (k_node - 1))


  def get_cluster_coefficient(self):
    sum = 0
    for node in self.nodes:
      cluster_coefficient = self.get_cluster_coefficient_node(node)
      sum += cluster_coefficient
    
    return sum / len(self.nodes)

# Graph Generation

In [26]:
import random

class ErdosRenyiGraph(Graph):
  def __init__(self, number_nodes, number_edges):
    super().__init__()
    self.number_nodes = number_nodes
    self.number_edges = number_edges

  def generate(self):
    for i in range(self.number_nodes):
      self.add_node(i)

    all_possible_edges = []
    for node1 in range(self.number_nodes):
      for node2 in range(node1+1, self.number_nodes):
        all_possible_edges.append(Edge(node1, node2))
    
    selected_edges = random.sample(all_possible_edges, self.number_edges)
    for edge in selected_edges:
      self.add_edge(edge)


In [37]:
import random

class SmallWorldGraph(Graph):
  def __init__(self, number_nodes, number_edges, random_prob):
    super().__init__()
    self.number_nodes = number_nodes
    self.number_edges = number_edges
    self.scale = self.number_edges / self.number_nodes
    self.random_prob = random_prob

  def random_change(self, number_random):
    remove_edges = random.sample(self.edges, number_random)
    source_edges_remove = list()
    for edge in remove_edges:
      self.remove_edge(edge)
      source_edges_remove.append(edge.source)

    source_edges_remove = random.sample(source_edges_remove, self.number_edges - self.get_number_edges())

    all_nodes = list(self.nodes)
    for node1 in source_edges_remove:
      while True:
        node2 = random.choice(all_nodes)
        edge = Edge(node1, node2)
        if node2 != node1 and not self.is_exist(edge):
          self.add_edge(edge)
          break

  def generate(self):
    for i in range(self.number_nodes):
      self.add_node(i)

    number_edges1 = int(self.scale)
    number_edges2 = int((self.scale - int(self.scale))*100 + 1)
    
    counter1 = 0
    counter2 = 0
    
    for node1 in range(self.number_nodes):
      for distance in range(1, number_edges1+1):
        node2 = (node1 + distance)%self.number_nodes
        self.add_edge(Edge(node1, node2))
        counter1 += 1

      if (node1 %100) < number_edges2:
        node2 = (node1 + number_edges1 + 1)%self.number_nodes
        self.add_edge(Edge(node1, node2))
        counter2 += 1

    number_random = int((self.get_number_edges() - self.number_edges) +\
                          self.random_prob * self.get_number_edges())
    
    self.random_change(number_random = number_random)

In [28]:
import pandas as pd

class LastfmGraph(Graph):
  def __init__(self):
    super().__init__()
    self._load_data()

  def _load_data(self):
    !gdown https://snap.stanford.edu/data/lastfm_asia.zip
    !unzip  -n /content/lastfm_asia.zip

    lastfm_dataset = pd.read_csv("/content/lasftm_asia/lastfm_asia_edges.csv")

    for i, data in lastfm_dataset.iterrows():
      node1 = data["node_1"]
      node2 = data["node_2"]

      self.add_edge(Edge(node1, node2))

# Create Graphs

In [29]:
lastfm_graph = LastfmGraph()

Downloading...
From: https://snap.stanford.edu/data/lastfm_asia.zip
To: /content/lastfm_asia.zip
100% 6.53M/6.53M [00:04<00:00, 1.47MB/s]
Archive:  /content/lastfm_asia.zip


In [31]:
er_graph = ErdosRenyiGraph(number_nodes=7624, number_edges=27806)

In [32]:
er_graph.generate()

In [38]:
small_world_graph = SmallWorldGraph(number_nodes=7624, number_edges=27806, random_prob=0.05)

In [39]:
small_world_graph.generate()

# Digree Distribution

# Clustering Coefficient

In [34]:
er_graph.get_cluster_coefficient()

0.0009161118793626045

In [40]:
small_world_graph.get_cluster_coefficient()

0.542798989369023

In [30]:
lastfm_graph.get_cluster_coefficient()

0.21941842432708525